In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf

# Data
import xarray as xr
import h5py
import pickle

# Helpful
import time
import datetime
import itertools
from itertools import product
from tqdm import tqdm
import os

# My Methods
import importlib
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
from src.models.EMOS_global.EMOS_global_load_models import *

2023-05-22 15:01:25.848776: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-05-22 15:01:25.848823: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

#### 0. Basics

In [4]:
var_names = ["u10", "v10", "t2m", "t850", "z500"]

<IPython.core.display.Javascript object>

In [5]:
# Variables to make testing easier --> determines variable and lead_time
var = 0  # 0,...,4
lead_time = 0  # 0,...,30

<IPython.core.display.Javascript object>

#### 1. Load Dataset

In [6]:
dat_train_proc = ldp.load_data_all_train_proc_norm()
dat_test_proc = ldp.load_data_all_test_proc_norm()

<IPython.core.display.Javascript object>

#### 2. Data Split

In [7]:
X_train_var_lead_all, y_train_var_lead_all = split_var_lead(dat_train_proc)
X_test_lead_all, y_test_var_lead_all = split_var_lead(dat_test_proc)

<IPython.core.display.Javascript object>

#### 3. Calculate CRPS Baseline

In [8]:
mu = X_test_lead_all[var][lead_time].isel(mean_std=0).values
sigma = X_test_lead_all[var][lead_time].isel(mean_std=1).values
y = y_test_var_lead_all[var][lead_time].values

<IPython.core.display.Javascript object>

In [10]:
X_test_lead_all[var][lead_time].shape

(357, 120, 130, 2)

<IPython.core.display.Javascript object>

In [9]:
combined_array = np.stack((mu, sigma), axis=1)

<IPython.core.display.Javascript object>

In [11]:
combined_array.shape

(357, 2, 120, 130)

<IPython.core.display.Javascript object>

In [16]:
crps_tensor = crps_cost_function_trunc(y, combined_array, theano=True)

<IPython.core.display.Javascript object>

In [13]:
crps_trunc_numpy = crps_trunc(mu=mu, sigma=sigma, y=y)

<IPython.core.display.Javascript object>

In [14]:
crps_trunc_numpy.mean()

0.50713784

<IPython.core.display.Javascript object>

In [15]:
crps_tensor

<tf.Tensor: shape=(), dtype=float32, numpy=0.50713766>

<IPython.core.display.Javascript object>